In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import RFE
import sklearn.feature_selection
import xgboost as xgb
from sklearn.metrics import roc_auc_score

from numpy import loadtxt
from xgboost import plot_importance
from matplotlib import pyplot

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
raw = pd.read_csv("raw.csv")

In [3]:
usedColumns = ['CIGEVER','ALCEVER','MJEVER', 'COCEVER', 'HEREVER', 'BOOKED', 'DRIVALC2']
usedFeatures = ['CIGEVER','ALCEVER','MJEVER', 'COCEVER', 'HEREVER', 'DRIVALC2']

# selection = raw[usedColumns]

# selection.is_copy = False


# raw[raw == 2] = 0
# raw[raw == 4] = 0
# raw[raw == 3] = 1
# raw[raw == 85] = 0
# raw[raw == 94] = 0
# raw[raw == 97] = 1 #Refused to answer = yes
# raw[raw == 98] = 1 #Left blank = yes

# s_train, s_test = train_test_split(selection, train_size = 0.5)
train, test = train_test_split(raw, train_size = 0.5)

Y_train = train[['BOOKED']]
# Y_test = s_test[['BOOKED']]

X_train = train[usedFeatures]
# X_test = s_test[usedFeatures]

X_train.head()

,CIGEVER,ALCEVER,MJEVER,COCEVER,HEREVER,DRIVALC2
17908,2,2,2,2,2,4
22391,2,2,2,2,2,4
54110,1,1,1,1,2,2
36510,1,1,1,2,2,2
2416,2,1,2,2,2,2


In [4]:
model = xgb.XGBRegressor(objective="binary:logistic")

In [5]:
# model.fit(X_train, Y_train)

In [6]:
# predictions = model.predict(X_test)
# roc_auc_score(Y_test, predictions)

Feature Selection

In [7]:
# estimator = SVR(kernel="linear")
# selector = RFE(estimator, 5, step=1)
# selector = selector.fit(X_train, Y_train)

# selector.ranking_

Predictive Power Ranking using RFE:
1) Heroin
2) Cigarettes
3) Drinking and Driving
4) Alcohol
5) Marijuana
6) Cocaine

In [8]:
# plot_importance(model, importance_type='weight')
# pyplot.show()

In [9]:
Y_train = train[['BOOKED']]
Y_train.is_copy = False

Y_train[Y_train == 2] = 0
Y_train[Y_train == 4] = 0
Y_train[Y_train == 3] = 1
Y_train[Y_train == 85] = 0
Y_train[Y_train == 94] = 0
Y_train[Y_train == 97] = 1 #Refused to answer = yes
Y_train[Y_train == 98] = 1 #Left blank = yes

iterations = 50
sliceIndex = int(3000/iterations)
feats= []
threshold = int(len(train)*.3)
count = 0

In [10]:
for i in range(1,iterations):
    X_train = train.iloc[:,list(range((i-1)*sliceIndex,i*sliceIndex))]
    model.fit(X_train, Y_train)
    rankings = model.booster().get_score(importance_type='gain')

    for key,value in rankings.items():
        if value > 50:
            counts = X_train[key].value_counts()
                
            try:
                condition1 = counts[99] < threshold
            except KeyError:
                try:
                    condition1 = counts[999] < threshold
                except KeyError:
                    try:
                        condition1 = counts[9999] < threshold
                    except:
                        condition1 = True
                        
            try:
                condition2 = counts[98] < threshold
            except KeyError:
                try:
                    condition2 = counts[998] < threshold
                except KeyError:
                    try:
                        condition2 = counts[9998] < threshold
                    except:
                        condition2 = True
            
            try:
                condition3 = counts[-9] < threshold
            except KeyError:
                condition3 = True
                        
            if(condition1 and condition2):
                print(str(count) + key)
                feats.append([key, value])
        count = count + 1

2SLTREC
4CIGARMFU
19CIG30AV
34CIG100LF
36CIGAGE
77COCYFU
85CRAKREC
104HALLREC
139HALYFU
156INHREC
164ANALMFU
168ANALYFU
185VICOLOR
196TRNEVER
200TRANYFU
242IRCIGRC
284IRMJRC
295IRHALRC
297IRCOCRC
329IICIGFM
332IRCOCFY
346IRMJFM
351IRHALFY
393IICRKAGE
402IRSLTAGE
411SMKFLAG
412CIGMON
451CIGFLAG
461MRJFLAG
476TOBMON
480TOBFLAG
498COCFLAG
500HALFLAG
512CIGALCMO
522CDUFLAG
529CDCGMO
572HYDCODOP
583BENZOS
608MJAFU
626FUHAL21
635CIGAFU
652CIGAVGM
656DCIGAFU
657FUCOC21
666FUSUM21
767COCNEEDL
809RSKMJREG
827STNEDL06
830CIGCRAGP
837MEDMJALL
845CIGFNSMK
846MEDMJYR
858CIGNMCHG
869CIGREGNM
875BLNTYFU
877CIGIRTBL
900MRJPHCTD
913COCPHCTD
957COCFMCTD
962HALFMCTD
981HALPHCTD
994TRNPHCTD
1018STMPHCTD
1045ANLPHCTD
1055IICGCRGP
1057IICGRGNM
1059IRCGNINF
1065IICGIRTB
1071IRCGRGNM
1079IRCGAVD
1090MMLSLBS
1091MMTLOZS
1093MMTJNTNM
1095MMLS5LBS
1096MMTRD30D
1102PROB
1111MMJNPCTB
1122MMTRADE
1125MMTSELL
1126MMFSELL
1130HALYLU
1156CIGAGLST
1161MMGSELL
1178COCMLU
1181METHYLU
1202ANLFFOSP
1238TRNFFLSP
1242MTHOTHS

In [11]:
feats.sort(key=lambda x: x[1], reverse=True)

In [12]:
feats

[['TXILALEV', 889.0065000000001],
 ['IICGRGNM', 672.8258],
 ['MMGSELL', 605.20567],
 ['MMTRD30D', 601.795],
 ['MMTSELL', 512.7955454545455],
 ['STMPHCTD', 430.30715999999995],
 ['MMLS5LBS', 366.9334],
 ['MMTLOZS', 360.897],
 ['IICGIRTB', 344.9593333333334],
 ['HALLREC', 316.9700555555555],
 ['MMJNPCTB', 301.494925],
 ['MMFSELL', 290.86396199999996],
 ['HALYFU', 278.45601076923083],
 ['COCPHCTD', 277.02990958333334],
 ['MMTJNTNM', 271.551],
 ['TRANYFU', 256.643994],
 ['CIGAGE', 256.62671534782606],
 ['CIGFLAG', 236.78228233333337],
 ['MRJPHCTD', 236.72027189189194],
 ['MMLSLBS', 233.11135999999996],
 ['YOSELL2', 232.49156571428574],
 ['MMTRADE', 231.71508571428572],
 ['COCNEEDL', 231.4307291590909],
 ['CADRKHER', 229.7407888571429],
 ['COCMLU', 225.97272857142852],
 ['MJAFU', 209.39004480769228],
 ['TRNFFOSP', 206.96974999999998],
 ['COCFLAG', 193.46181933333327],
 ['BLNTYFU', 192.48560571428567],
 ['ANLFFOSP', 178.98698181818185],
 ['IICIGFM', 172.26206794117647],
 ['IRCOCRC', 168.2507

In [13]:
lst2 = [item[0] for item in feats]
#lst2.remove('CIGEVER')
#lst2.remove('QUESTID2')

In [14]:
lst2 = lst2[:150]

In [15]:
lst2

['TXILALEV',
 'IICGRGNM',
 'MMGSELL',
 'MMTRD30D',
 'MMTSELL',
 'STMPHCTD',
 'MMLS5LBS',
 'MMTLOZS',
 'IICGIRTB',
 'HALLREC',
 'MMJNPCTB',
 'MMFSELL',
 'HALYFU',
 'COCPHCTD',
 'MMTJNTNM',
 'TRANYFU',
 'CIGAGE',
 'CIGFLAG',
 'MRJPHCTD',
 'MMLSLBS',
 'YOSELL2',
 'MMTRADE',
 'COCNEEDL',
 'CADRKHER',
 'COCMLU',
 'MJAFU',
 'TRNFFOSP',
 'COCFLAG',
 'BLNTYFU',
 'ANLFFOSP',
 'IICIGFM',
 'IRCOCRC',
 'MEDMJALL',
 'COCYFU',
 'IRCIGRC',
 'SNFAMJEV',
 'IRHALFY',
 'TXNDILAL',
 'IRMJRC',
 'IRCOCFY',
 'HYDCODOP',
 'MRJFLAG',
 'COCFMCTD',
 'IICGCRGP',
 'TOBFLAG',
 'SPINVST2',
 'STNEDL06',
 'COCYRBFR',
 'HALPHCTD',
 'TRNEVER',
 'HPDRGTLK',
 'VICOLOR',
 'TRNPHCTD',
 'IRCGAVD',
 'MTHOTHS2',
 'ANLPHCTD',
 'AUALTYR',
 'HOMEVST',
 'CIGCRAGP',
 'LMMIYRU',
 'PROB',
 'AUINPYR',
 'METHYLU',
 'CIGAFU',
 'DCIGAFU',
 'CDCGMO',
 'FUSUM21',
 'CIG100LF',
 'ANALYFU',
 'CIGMON',
 'SLTREC',
 'TOBMON',
 'HALFMCTD',
 'IRCGNINF',
 'IICRKAGE',
 'RSKMJREG',
 'FUHAL21',
 'IRMJFM',
 'IRCGRGNM',
 'TXGPILAL',
 'CIGFNSMK',
 'SMKFL

In [16]:
minXTrain = train[lst2]
minXTest = test[lst2]

In [17]:
Y_test = test[['BOOKED']]
Y_test.is_copy = False
Y_test[Y_test == 2] = 0
Y_test[Y_test == 4] = 0
Y_test[Y_test == 3] = 1
Y_test[Y_test == 85] = 0
Y_test[Y_test == 94] = 0
Y_test[Y_test == 97] = 1 #Refused to answer = yes
Y_test[Y_test == 98] = 1 #Left blank = yes

In [18]:
model = xgb.XGBRegressor(n_estimators=100, objective="binary:logistic")
model.fit(minXTrain, Y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [21]:
predictions = model.predict(minXTest)
roc_auc_score(Y_test, predictions)

0.86681470187453656


In [20]:
minXTrain.to_csv("X_Train.csv")
minXTest.to_csv("X_Test.csv")
Y_train.to_csv("Y_Train.csv")
Y_test.to_csv("Y_Test.csv")